In [1]:
import awswrangler as wr
import pandas as pd
import boto3
import json

In [2]:
boto3.setup_default_session(profile_name='pawprint')

In [17]:
df = wr.s3.read_json('s3://pawprint-raw/genome/dataset_reports/date=2023-11-28/', lines=True)

In [22]:
final_list = []
for i in df['reports']:
    final_list.extend(i)


In [23]:
df = pd.json_normalize(final_list)

In [24]:
df.head()

,accession,current_accession,paired_accession,source_database,organelle_info,organism.tax_id,organism.organism_name,organism.infraspecific_names.isolate,organism.infraspecific_names.sex,assembly_info.assembly_level,...,organism.common_name,assembly_info.linked_assemblies,assembly_info.diploid_role,assembly_stats.total_number_of_chromosomes,assembly_info.paired_assembly.only_genbank,assembly_info.paired_assembly.only_refseq,assembly_info.genome_notes,assembly_info.suppression_reason,assembly_info.synonym,organism.infraspecific_names.breed
0,GCA_014108235.1,GCA_014108235.1,GCF_014108235.1,SOURCE_DATABASE_GENBANK,"[{'description': 'Mitochondrion', 'total_seq_l...",51298,Myotis myotis,mMyoMyo1,female,Scaffold,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GCF_014108235.1,GCF_014108235.1,GCA_014108235.1,SOURCE_DATABASE_REFSEQ,"[{'description': 'Mitochondrion', 'total_seq_l...",51298,Myotis myotis,mMyoMyo1,female,Scaffold,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GCA_028538775.1,GCA_028538775.1,NaN,SOURCE_DATABASE_GENBANK,"[{'description': 'Mitochondrion', 'total_seq_l...",159337,Myotis yumanensis,MYYU_CA2020_CCGP,male,Scaffold,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GCA_004026985.1,GCA_004026985.1,NaN,SOURCE_DATABASE_GENBANK,NaN,51298,Myotis myotis,US100,NaN,Scaffold,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GCA_028536395.1,GCA_028536395.1,NaN,SOURCE_DATABASE_GENBANK,NaN,159337,Myotis yumanensis,MYYU_CA2020_CCGP,male,Scaffold,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
df[~df['assembly_info.linked_assemblies'].isna()]['assembly_info.linked_assemblies'].head()

5     [{'linked_assembly': 'GCA_903992515.1', 'assem...
6     [{'linked_assembly': 'GCA_949987765.1', 'assem...
7     [{'linked_assembly': 'GCA_963455325.1', 'assem...
11    [{'linked_assembly': 'GCA_903992545.1', 'assem...
13    [{'linked_assembly': 'GCA_949987585.1', 'assem...
Name: assembly_info.linked_assemblies, dtype: object

In [38]:
len(df.columns.tolist())

90

In [42]:
expectations = [
    {
        "expectation_type": "expect_column_values_to_be_of_type",
        "kwargs": {
            "column": column,
            "type_": str(df[column].dtype)
        }
    }
    for column in df.columns
]
json.dump(expectations, open('Pawprint/config/expectations.json', 'w'))

In [32]:
from great_expectations.data_context import BaseDataContext
from great_expectations.data_context.types.base import DataContextConfig, DatasourceConfig, AnonymizedUsageStatisticsConfig
from great_expectations.dataset import PandasDataset
import great_expectations as gx

context = gx.get_context(
    DataContextConfig(
        datasources={
            "pandas_datasource": DatasourceConfig(
                class_name="PandasDatasource"                
            )
        },
        stores={
            "expectations_store": {
                "class_name": "ExpectationsStore",
                "store_backend": {
                    "class_name": "TupleFilesystemStoreBackend",
                    "base_directory": "/Volumes/GeekFoxLab/Projetos/Pawprint/config/expectations",
                }
            }
        },
        expectations_store_name="expectations_store",
    )
)

In [50]:
suite = context.get_expectation_suite('genome.dataset_report')
batch = PandasDataset(df, expectation_suite=suite)
results = batch.validate()
results

{
  "success": true,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "expectation_type": "expect_table_columns_to_match_ordered_list",
        "kwargs": {
          "column_list": [
            "accession",
            "current_accession",
            "paired_accession",
            "source_database",
            "organelle_info",
            "organism.tax_id",
            "organism.organism_name",
            "organism.infraspecific_names.isolate",
            "organism.infraspecific_names.sex",
            "assembly_info.assembly_level",
            "assembly_info.assembly_status",
            "assembly_info.paired_assembly.accession",
            "assembly_info.paired_assembly.status",
            "assembly_info.paired_assembly.annotation_name",
            "assembly_info.paired_assembly.changed",
            "assembly_info.assembly_name",
            "assembly_info.assembly_type",
            "assembly_info.bioproject_lineage",
            "assembl